# VacationPy
----


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_data = pd.read_csv("../output_data/clean_city_data.csv")

# Visualize
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,53.01,89,100,2.73,US,1602861190
1,bluff,-46.60,168.33,42.01,85,23,1.01,NZ,1602861006
2,rikitea,-23.12,-134.97,73.26,80,99,18.37,PF,1602861191
3,kapaa,22.08,-159.32,75.20,94,75,8.05,US,1602861191
4,petropavlovsk-kamchatskiy,53.05,158.65,32.00,81,40,2.24,RU,1602860976


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
city_locations = city_data[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
humidity = city_data["Humidity"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure(center=(10,20), zoom_level=2, \
                   layout={'width': '800px', 'margin': '0 auto 0 auto'})


heatmap_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3, opacity=0.5)


fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='800px'))

### Heat map that displays the humidity for every city
Screenshot

![Hotels in ideal Weather](../output_data/weather_heatmap.png)

### Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions.
ideal weather conditions:

   * A max temperature lower than 80 degrees but higher than 70.
   * Wind speed less than 10 mph.
   * Zero cloudiness.

In [6]:
ideal_city = city_data.loc[(city_data['Max Temp'] > 70) & 
                           (city_data['Max Temp'] < 80) &
                           (city_data['Wind Speed'] < 10) &
                           (city_data['Cloudiness'] == 0)
                          ].reset_index(drop=True)
ideal_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,general roca,-39.03,-67.58,75.20,47,0,3.36,AR,1602861195
1,saldanha,-33.01,17.94,77.00,33,0,9.17,ZA,1602860958
2,guerrero negro,27.98,-114.06,74.64,60,0,9.95,MX,1602861205
3,castro,-24.79,-50.01,70.68,55,0,7.94,BR,1602861209
4,abu samrah,35.30,37.18,72.68,30,0,4.52,SY,1602861219
5,puntarenas,9.98,-84.84,77.00,65,0,8.05,CR,1602861060
6,abha,18.22,42.51,75.20,10,0,9.17,SA,1602861229
7,harsin,34.27,47.59,71.60,2,0,6.93,IR,1602861231
8,fasa,28.94,53.65,73.40,9,0,6.93,IR,1602861238
9,alice springs,-23.70,133.88,75.20,31,0,5.82,AU,1602861260


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame(columns=['Hotel Name'])
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_city.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request and print url
    hotel_results = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:   
        hotel_df.loc[index, "Hotel Name"] = hotel_results["results"][0]["name"]
        hotel_df.loc[index, "Lat"] = hotel_results["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = hotel_results["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "Address"] = hotel_results["results"][0]["vicinity"]
        print(f"Retieved information of Hotel {hotel_results['results'][0]['name']}.")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# Set CSS properties for th elements in dataframe
th_props = [('border', 'thin solid'),
            ('text-align', 'left'),
            ('background-color', '#dfdfdf')]

# output the dataframe
hotel_df.style.set_properties(**{'text-align': 'left'}) \
              .set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])

Retieved information of Hotel Hotel Huemul.
Retieved information of Hotel Protea Hotel by Marriott Saldanha Bay.
Retieved information of Hotel Hotel Los Caracoles.
Retieved information of Hotel Urano Business Hotel.
Missing field/result... skipping.
Retieved information of Hotel Hotel Yadran Beach Resort & Casino.
Retieved information of Hotel Sarawat Park Hotel & Resort.
Missing field/result... skipping.
Retieved information of Hotel Jahangardi Hotel.
Retieved information of Hotel DoubleTree by Hilton Hotel Alice Springs.
Retieved information of Hotel Café Colonial - Hotel Kindermann.
Retieved information of Hotel ibis Styles Mt Isa Verona.
Retieved information of Hotel Cape Vermeer.
Retieved information of Hotel Hotel Puerto Juan Montiel SPA & Base Náutica.


,Hotel Name,Lat,Lng,Address
0,Hotel Huemul,-39.029201,-67.571607,"Gral. Manuel Belgrano 1479, FEE"
1,Protea Hotel by Marriott Saldanha Bay,-33.009115,17.947599,"51 Main Rd, Saldanha"
2,Hotel Los Caracoles,27.967758,-114.028336,"Calzada, De La República s/n, Estado 30, Guerrero Negro"
3,Urano Business Hotel,-24.803363,-50.011850,"R. José Marques, 26 - Vila Frei Mathias, Castro"
5,Hotel Yadran Beach Resort & Casino,9.975638,-84.849391,"Paseo de los Turistas, Carmen, Puntarenas"
6,Sarawat Park Hotel & Resort,18.199748,42.497364,"6169 King Abdul Aziz Rd, 2970, Abha"
8,Jahangardi Hotel,28.943129,53.639358,Fasa
9,DoubleTree by Hilton Hotel Alice Springs,-23.715016,133.879153,"82 Barrett Dr, Alice Springs"
10,Café Colonial - Hotel Kindermann,-26.781134,-51.016579,"Rua José Bonifácio, 13 - Subsolo - Paraíso, Caçador"
11,ibis Styles Mt Isa Verona,-20.726140,139.492803,"Cnr Rodeo Dr And, Camooweal St, Mount Isa"


In [10]:
hotel_df[["City","Country"]] = ideal_city[["City","Country"]]
hotel_df.style.set_properties(**{'text-align': 'left'}) \
              .set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])

,Hotel Name,Lat,Lng,Address,City,Country
0,Hotel Huemul,-39.029201,-67.571607,"Gral. Manuel Belgrano 1479, FEE",general roca,AR
1,Protea Hotel by Marriott Saldanha Bay,-33.009115,17.947599,"51 Main Rd, Saldanha",saldanha,ZA
2,Hotel Los Caracoles,27.967758,-114.028336,"Calzada, De La República s/n, Estado 30, Guerrero Negro",guerrero negro,MX
3,Urano Business Hotel,-24.803363,-50.011850,"R. José Marques, 26 - Vila Frei Mathias, Castro",castro,BR
5,Hotel Yadran Beach Resort & Casino,9.975638,-84.849391,"Paseo de los Turistas, Carmen, Puntarenas",puntarenas,CR
6,Sarawat Park Hotel & Resort,18.199748,42.497364,"6169 King Abdul Aziz Rd, 2970, Abha",abha,SA
8,Jahangardi Hotel,28.943129,53.639358,Fasa,fasa,IR
9,DoubleTree by Hilton Hotel Alice Springs,-23.715016,133.879153,"82 Barrett Dr, Alice Springs",alice springs,AU
10,Café Colonial - Hotel Kindermann,-26.781134,-51.016579,"Rua José Bonifácio, 13 - Subsolo - Paraíso, Caçador",cacador,BR
11,ibis Styles Mt Isa Verona,-20.726140,139.492803,"Cnr Rodeo Dr And, Camooweal St, Mount Isa",mount isa,AU


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(10,20), zoom_level=2, \
                   layout={'width': '800px', 'margin': '0 auto 0 auto'})
heatmap_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3, opacity=0.5)


fig.add_layer(heatmap_layer)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='800px'))

### Holtel Suggestions for Vacation planning

In cities with ideal weather conditions:

* A max temperature lower than 80 degrees but higher than 70.

* Wind speed less than 10 mph.

* Zero cloudiness.

Screenshot

![Hotels in ideal Weather](../output_data/hotels_at_ideal_weather_map.png)